# CHANGE FILE PATHS EVERYWHERE (Reduce, Plot, Generate, Correlations) TO INCLUDE NIGHT OF OBSERVATION

# Imports

In [1]:
# Add the directory where starships' directory is located
from sys import path
path.append('/home/mathisb/Github/')

# Add the directory containing the input data (opacity, abundance and stellar specs files)
import os
os.environ['pRT_input_data_path'] = '/home/mathisb/projects/def-ncowan/input_data'

In [2]:
from pathlib import Path

import matplotlib.pyplot as plt
import starships.homemade as hm
from starships import spectrum as spectrum
from starships.mask_tools import interp1d_masked

interp1d_masked.iprint=False
import warnings

import astropy.constants as const
import astropy.units as u
import numpy as np
import starships.planet_obs as pl_obs
import starships.plotting_fcts as pf
from starships.planet_obs import Observations

warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", RuntimeWarning)

# Read spectra and useful quantities

## Note: Better to use the scratch space to save the reductions.
You have infinite space, but the files are deleted if untouched for 2 months. It allows to save as many reductions as desired. Once the correct reduction parameters are set, you can move them into your home directory.

In [3]:
#--------------------------------------------------------------------------------------------
# Planet, night, reduction and instrument
pl_name = 'TRAPPIST-1 b'
visit_name = '2019-06-14'
reduction = 'v001'
instrument = "SPIRou"

#--------------------------------------------------------------------------------------------

# Directory where to save reductions
# > use scratch if available, use home if not
try:
    out_dir = Path(os.environ['SCRATCH'])
except KeyError:
    out_dir = Path.home()

# Output reductions in dedicated directory
pl_name_fname = ''.join(pl_name.split())
out_dir /= Path(f'HRS_reductions/{instrument}/{pl_name_fname}/{visit_name}/{reduction}')
# Make sure the  directory exists
out_dir.mkdir(parents=True, exist_ok=True)


In [4]:
# Relevant directories

# Where to find the observations (data)?
obs_dir = f"/home/mathisb/Github/HRS_data/{instrument}/TRAPPIST-1/{visit_name}/"  # for TRAPPIST-1 SPIRou data

# Where to save figures?
path_fig = Path(f'HRS_reductions/{instrument}/{pl_name_fname}/{visit_name}/Figures')

In [5]:
# Get all the filenames for e2ds, tcorr and recon files. Copy these in text files to be used in the cell below.

# # ed2s
# for i in sorted(os.listdir(obs_dir)):
#     if i[-6:] == "e.fits":
#         print(i)

# # tcorr
# for i in sorted(os.listdir(obs_dir)):
#     if i[-6:] == "t.fits":
#         print(i)

# # recon
# for i in sorted(os.listdir(obs_dir)):
#     if i[-10:] == "RECON.fits":
#         print(i)


In [6]:
# All the observations must be listed in files.
# We need the e2ds, the telluric corrected and the reconstructed spectra.

list_filenames = {'list_e2ds': 'list_e2ds.txt',
                  'list_tcorr': 'list_tcorr.txt',
                  'list_recon': 'list_recon.txt'}

In [7]:
# SPECIFY PARAMETERS OF THE PLANET
# If not specified, the default parameters from exofile are taken.

# Most parameters from trappist.one website
ap     = 0.01150*u.au       # semi-major axis of planet
R_star = 0.121*u.R_sun      # radius of star
R_pl   = 1.127*u.R_earth    # radius of planet
M_star = 0.089*const.M_sun  # mass of star
e      = 0                  # eccentricity
w      = 4.712389*u.rad     # argument of periapsis (equal to 270 degrees)

# Mid-transit time (BJD) of this visit
mid_tr = 2458649.07123219*u.d  # calculated from Agol et al. 2021


#--------------------------------------------------------------------------------------------


obs = Observations(name=pl_name, 
                   pl_kwargs = {'M_star': M_star, 
                                'R_star': R_star,
                                'ap': ap,
                                'R_pl': R_pl,
                                'mid_tr': mid_tr,
                                'excent': e,
                                'w': w})

# Get the data
obs.fetch_data(obs_dir, **list_filenames)
p = obs.planet

# Verify that the info is good
print("M_star:", M_star, "\t", "R_star:", R_star, "\nap:", ap, "\t", "R_pl:", R_pl,  "\t", "mid_tr:", mid_tr, 
      "\ne:", e,  "\t", "w:", w)

Getting TRAPPIST-1 b from ExoFile


INFO:starships.planet_obs:Fetching the uncorrected spectra
INFO:starships.planet_obs:File: list_e2ds.txt


Changing M_star from [1.5907279e+29] kg to 1.769684784921265e+29 kg
It became [1.76968478e+29] kg
Changing R_star from [0.] m to 0.121 solRad
It became [84179700.] m
Changing ap from [1.72546184e+09] m to 0.0115 AU
It became [1.72037551e+09] m
Changing R_pl from [6934724.] m to 1.127 earthRad
It became [7188118.7] m
Changing mid_tr from [2457322.514193] d to 2458649.07123219 d
It became [2458649.07123219] d
Changing excent from pl_orbeccen
-----------
    0.00622 to 0
It became 0
Changing w from [10.5917051] rad to 4.712389 rad
It became [4.712389] rad


IndexError: list index out of range

# Build transmission spectrum

In [22]:
"""
param_all: Reduction parameters
telluric fraction to mask (usually varied between 0.2 and 0.5), 
limits for the wings, 
width of the smoothing kernel for the low pass filter (fixed at 51), 
useless param, 
width of the gaussian kernel for low pass filter (fixed at 5),
nPC (nb of principal components) to remove (varied between 1 and 5),
sigma clips params (fixed at 5.0)
"""

#--------------------------------------------------------------------------------------------
# PARAMETERS TO CHANGE

RVsys = [-52.003101]  # change to RV of star system
kind_trans = 'transmission'  # emission or transmission
coeffs = [0.02703969,  1.10037972, -0.96372403,  0.28750393]  # limb darkening coefficients
ld_model = 'nonlinear'


tellu_frac = 0.45  # telluric fraction to mask (usually varied between 0.2 and 0.5)
nPC = [1]  # number of principal components to remove

mask_wings = 0.97  # fraction of wings of deep tellurics that is masked

#--------------------------------------------------------------------------------------------

cbp = True  # correct bad pixels

iout_all = ['all']
polynome = [False] 
do_tr = [1]
transit_tags = [None]  # use if want to remove spectra

kwargs_gen_tr = {
    'coeffs' : coeffs,
    'ld_model' : ld_model,
    'do_tr' : do_tr,
    'kind_trans' : kind_trans,
    'polynome' : polynome,
    'cbp': cbp }

kwargs_build_ts = {
    'clip_ratio' : 6,
    'clip_ts' : 6,
    'unberv_it' : True }

for n_pc in nPC:
    params_all=[[tellu_frac, mask_wings, 51, 41, 5, n_pc, 5.0, 5.0, 5.0, 5.0]]
    list_tr = pl_obs.generate_all_transits(obs, transit_tags, RVsys, params_all, iout_all,
                                           **kwargs_gen_tr, **kwargs_build_ts)

    # Save sequence with all reduction steps
    out_filename = f'sequence_{n_pc}-pc_mask_wings{mask_wings*100:n}'
    pl_obs.save_single_sequences(out_filename, list_tr['1'], path=out_dir,
                                 filename_end=visit_name, save_all=True)
    
    # Save sequence with only the info needed for a retrieval (to compute log likelihood).
    out_filename = f'retrieval_input_{n_pc}-pc_mask_wings{mask_wings*100:n}'
    pl_obs.save_sequences(out_filename, list_tr, do_tr, path=out_dir)

Transmission
Masking high variance pixels (quick fix for OH lines). 
flux_norm all nan : False
Shifting everything in the stellar ref. frame and normalizing by the median 
Spectra 
 Unberv : 48 - 15  
Telluriques 
 Unberv : 48 - 15  
flux_Sref all nan : False
Masking deep tellurics. 
flux_masked all nan : False
Building the master out #1 
ratio_filt has values <= 0.75!
flux_norm_mo all nan : False
master_out all nan : False
Building the transmission spectrum #1 
spec-trans all nan : False
Removing the static noise with PCA and sigma cliping 
(16, 49, 4088)
spec_trans all nan : False
clean_ts all nan : False
Removing the mean 
Removing the remaining high variance pixels. 

Removing the mean. 
Calculating noise with 1 PCs
/scratch/mathisb/HRS_reductions/SPIRou/TRAPPIST-1b/v011/sequence_1-pc_mask_wings97_data_trs_night1.npz
/scratch/mathisb/HRS_reductions/SPIRou/TRAPPIST-1b/v011/retrieval_input_1-pc_mask_wings97_data_info.npz
/scratch/mathisb/HRS_reductions/SPIRou/TRAPPIST-1b/v011/retriev

## That's it!

In [23]:
# Verify if every parameter makes sense

print("Mid-transit time (BJD):", "\t", p.mid_tr,
      "\nRadius of the star:", "\t\t", p.R_star.to(u.R_sun),
      "\nRadius of the planet:", "\t\t", p.R_pl.to(u.R_earth),
      "\nObservations in transit:", "\t", obs.iIn,
      "\nObservations out of transit:", "\t", obs.iOut,
      "\nSeparation between planet and star:", "\n", (obs.sep / p.R_star), "R_star"
     )

Mid-transit time (BJD): 	 [2459514.80598483] d 
Radius of the star: 		 [0.121] solRad 
Radius of the planet: 		 [1.127] earthRad 
Observations in transit: 	 [ 7  8  9 10] 
Observations out of transit: 	 [ 0  1  2  3  4  5  6 11 12 13 14 15] 
Separation between planet and star: 
 [4.50230374 3.98842203 3.47224653 2.95415633 2.43496409 1.91542418
 1.39741032 0.88611543 0.4119805  0.32794193 0.77392786 1.28170719
 1.79893794 2.31844711 2.83771907 3.35610423] R_star
